In [53]:
# basic modules
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.metrics import accuracy_score

In [2]:
train_data = pd.read_csv("/Users/lippek/Desktop/ML in R/Python/Zadania/data/Ridge_regression_houseprice/train.csv")
test_data = pd.read_csv("/Users/lippek/Desktop/ML in R/Python/Zadania/data/Ridge_regression_houseprice/test.csv")

x = train_data.drop(['SalePrice', 'Id'], axis = 1)
train_labels = train_data['SalePrice'].values
x_test = test_data.drop(['Id'], axis = 1)

In [3]:
# Typeee of each column
for colname in x.columns:
    print(colname + ':' +str(x[colname].dtype))

MSSubClass:int64
MSZoning:object
LotFrontage:float64
LotArea:int64
Street:object
Alley:object
LotShape:object
LandContour:object
Utilities:object
LotConfig:object
LandSlope:object
Neighborhood:object
Condition1:object
Condition2:object
BldgType:object
HouseStyle:object
OverallQual:int64
OverallCond:int64
YearBuilt:int64
YearRemodAdd:int64
RoofStyle:object
RoofMatl:object
Exterior1st:object
Exterior2nd:object
MasVnrType:object
MasVnrArea:float64
ExterQual:object
ExterCond:object
Foundation:object
BsmtQual:object
BsmtCond:object
BsmtExposure:object
BsmtFinType1:object
BsmtFinSF1:int64
BsmtFinType2:object
BsmtFinSF2:int64
BsmtUnfSF:int64
TotalBsmtSF:int64
Heating:object
HeatingQC:object
CentralAir:object
Electrical:object
1stFlrSF:int64
2ndFlrSF:int64
LowQualFinSF:int64
GrLivArea:int64
BsmtFullBath:int64
BsmtHalfBath:int64
FullBath:int64
HalfBath:int64
BedroomAbvGr:int64
KitchenAbvGr:int64
KitchenQual:object
TotRmsAbvGrd:int64
Functional:object
Fireplaces:int64
FireplaceQu:object
GarageTy

In [4]:
# fucntion to changeing column type
def prepro_change_column_type(x):
    x['MSSubClass'] = x['MSSubClass'].astype('str')

In [5]:
prepro_change_column_type(x)

In [6]:
prepro_test_list = []
prepro_test_list.append(prepro_change_column_type)

In [7]:
# remove columns with moree than 50% missing values
count_null = x.isnull().sum(axis = 0)/x.shape[0]
variable_null = count_null[count_null > 0]

In [8]:
# removee variable with more than 50% missing valuees
remove_variable_index = list(variable_null[variable_null > 0.5].index)

In [9]:
variable_null.drop(remove_variable_index, inplace = True)

In [10]:
def prepro_nan_columns(x):
    x.drop(remove_variable_index, axis = 1, inplace = True)
prepro_nan_columns(x)

In [11]:
print('variablee with NaN after droping those with moree than 50% missing:', variable_null)

variablee with NaN after droping those with moree than 50% missing: LotFrontage     0.177397
MasVnrType      0.005479
MasVnrArea      0.005479
BsmtQual        0.025342
BsmtCond        0.025342
BsmtExposure    0.026027
BsmtFinType1    0.025342
BsmtFinType2    0.026027
Electrical      0.000685
FireplaceQu     0.472603
GarageType      0.055479
GarageYrBlt     0.055479
GarageFinish    0.055479
GarageQual      0.055479
GarageCond      0.055479
dtype: float64


In [12]:
# choose cat and num columns 
num_columns = x.select_dtypes(include = np.number).columns.tolist()
cat_columns = x.select_dtypes(include = ['object', 'category']).columns.tolist()

In [13]:
# remove observations with moree than 50% missing values 
count_obs_null = x.isnull().sum(axis = 1)/x.shape[1]
obs_null = count_obs_null[count_obs_null > 0]
remove_obs_index = list(obs_null[obs_null > 0.5].index)
x.drop(remove_obs_index, axis = 1, inplace = True)
print(len(remove_obs_index), 'observations reemoved becausee of having more than 50% of null values')

0 observations reemoved becausee of having more than 50% of null values


In [14]:
# imptuding missing values with from thee knowledge of this paritcualr dataset
def prepro_nan_objective_imputing(x):
    x['MasVnrType'].fillna('None', inplace = True)
    
    aux_list = ['BsmtQual',
                'BsmtCond',
                'BsmtExposure',
                'BsmtFinType1',
                'BstmFinType2',
                'GarageType',
                'GarageFinish',
                'GarageQual',
                'GarageCond',
                'FireplaceQu'
        
    ]
    for i in aux_list:
        x[i].fillna('NA', inplace = True)
        
    #numerical
    x['MasVnrArea'].fillna(0, inplace = True)
    
    x.loc[x['BsmtCond'] == 'NA', ['BsmtUnfSF',
                                   'TotalBsmtSF',
                                   'BsmtFullBath',
                                   'BsmtHalfBath',
                                   'BsmtFinSF1',
                                   'BsmtFinSF2']]=0
    prepro_nan_obejctive_imputing(x)
    prepro_test_list.append(prepro_nan_objective_imputing)

In [15]:
# removing outlier
x = x.loc[
    (x['LotArea'] < 100000)
    | (x['LotFrontage'] < 250)
    | (x['1stFlrSF'] < 4000)
    | (x['BsmtFinSF1'] < 5000)
    | (x['BsmtFinSF2'] < 1400)
    | (x['EnclosedPorch'] < 500)
    | (x['GrLivArea'] < 5000)
    | (x['TotalBsmtSF'] < 6000),
]

In [16]:
x.shape

(1460, 75)

In [17]:
# Automatic missing values imputation and normalization in numerical variables using meedian and mode
# After that Wee will standardzie the numeric variables and one-hot eencode thee categorical ones

In [18]:
numeric_transform = Pipeline([
    ('median', SimpleImputer(strategy = 'median')),
    ('standard', StandardScaler())
])
categorical_transform = Pipeline([
    ('mostfreq', SimpleImputer(strategy = 'most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
])

In [19]:
prepro_transformer = ColumnTransformer([('num', numeric_transform, num_columns),
                                       ('cat', categorical_transform, cat_columns)])

## 2. Modeling

In [32]:
y = np.log10(train_labels)

In [23]:
pipeline = Pipeline([('prepro', prepro_transformer), ('estimator', Ridge())])

In [33]:
# apply Grid Search to find optimal hyperparametrs for the algorithm
param_grid = dict(estimator__alpha = np.linspace(10, 100, 100))
grid_search = GridSearchCV(pipeline, param_grid, scoring = 'neg_mean_squared_error', cv = 5, verbose = 0)

In [34]:
grid_search.fit(x, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('prepro',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('median',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standard',
                                                                                          StandardScaler())]),
                                                                         ['LotFrontage',
                                                                          'LotArea',
                                                                          'OverallQual',
                                                                          'OverallCond',
                                                             

In [36]:
best_model = grid_search.best_estimator_
best_model

Pipeline(steps=[('prepro',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('median',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standard',
                                                                   StandardScaler())]),
                                                  ['LotFrontage', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBuilt', 'YearRemodAdd',
                                                   'MasVnrArea', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'BsmtUnfSF',
                                                   'TotalBsmtSF', '1stFlrSF',
                                                   '2ndFlrSF', 'LowQualFinSF',
                  

## 3. Predicting 

In [37]:
for func in prepro_test_list:
    func(x_test)

In [39]:
y_pred = 10**best_model.predict(x_test)

In [45]:
df_submit = pd.DataFrame({'Id': test_data['Id'], 'SalePrice': c})
df_submit

,Id,SalePrice
0,1461,113453.522679
1,1462,145597.228297
2,1463,169815.073093
3,1464,192413.206242
4,1465,199798.557337
...,...,...
1454,2915,85730.527668
1455,2916,83958.994492
1456,2917,156863.359919
1457,2918,114138.106217
